In [ ]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from urllib import request
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms

from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
# 이미지출처: https://www.green-dog.com/cocokara/issho-ni-motto/about-the-wisdom-of-a-dog-230/
url = 'https://www.green-dog.com/cocokara/wp/wp-content/uploads/7564-00230_3.jpg'        

In [ ]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, 'data')
os.makedirs(data_dir, exist_ok=True)

In [ ]:
save_fname = os.path.join(data_dir, 'dogs.jpg')
request.urlretrieve(url, save_fname)

In [ ]:
img = Image.open(save_fname).convert("RGB")

In [ ]:
img

In [ ]:
class Image_Transform(object):
    def __init__(self):
        self.transform_shape = self.get_pil_transform()
        self.transform_tensor = self.get_preprocess_transform()
    
    @staticmethod
    def get_pil_transform(): 
        transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224)
        ])    
        return transform   
    
    @staticmethod
    def get_preprocess_transform():
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])     
        transform = transforms.Compose([
            transforms.ToTensor(),
            normalize
        ])    
        return transform
    
    @staticmethod
    def transform_unsqueeze(img):
        transform = get_input_transform()
        # unsqeeze converts single image to batch of 1
        return transform(img).unsqueeze(0)
    
img_transform = Image_Transform()

In [ ]:
sample_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
])

In [ ]:
# 사전학습된 모델 호출
model = models.inception_v3(pretrained=True)

In [ ]:
# ImageNet class index 다운로드 주소
# https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json

# JSON 파일 호출
idx2label, cls2label, cls2idx = [], {}, {}
with open(file=os.path.abspath('./data/imagenet_class_index.json'), mode='rt') as read_file:
    class_idx = json.load(read_file)
    idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
    cls2label = {class_idx[str(k)][0]: class_idx[str(k)][1] for k in range(len(class_idx))}
    cls2idx = {class_idx[str(k)][0]: k for k in range(len(class_idx))}

In [ ]:
# 불러온 이미지를 텐서 형태로 만든다
img_t = sample_transform(img).unsqueeze(0)

In [ ]:
# 사전학습된 모델을 검증 모드로 바꾼다
# Dropout 제거 및 BatchNorm의 파라미터 고정 
model.eval()

# 이미지로부터 예측된 각 클래스 별 로짓 값을 산출
logits = model(img_t)

In [ ]:
logits.float()

In [ ]:
# 각 로짓 값을 0~1 사이의 확률값으로 변환
probs = F.softmax(logits, dim=1)

# 그 중 확률값이 가장 높은 5개를 반환
probs5 = probs.topk(5)

In [ ]:
probs

In [ ]:
# 반환된 5개의 클래스 별 확률값 및 인덱스/인덱스명 시각화
array_prob = probs5[0][0].detach().numpy()
array_class = probs5[1][0].detach().numpy()
col = ['Probability', 'Index_number', 'Index_name']
pd.DataFrame(tuple((p,c, idx2label[c]) for p, c in zip(array_prob, array_class)), columns=col)

In [ ]:
# 예측한 클래스 인덱스 값 반환 (최대 로짓을 가지는 클래스의 인덱스 값 반환)
def batch_predict(images):
    model.eval()
    batch = torch.stack(tuple(img_transform.transform_tensor(i) for i in images), dim=0)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    batch = batch.to(device)
    
    logits = model(batch)
    probs = F.softmax(logits, dim=1)
    return probs.detach().cpu().numpy()

In [ ]:
# 입력할 이미지를 리스트 배열 안에 넣는다
img_lst = [img_transform.transform_shape(img)]

test_pred = batch_predict(img_lst)
test_pred.squeeze().argmax()

In [ ]:
img_shaped = img_transform.transform_shape(img)

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(np.array(img_shaped), 
                                         batch_predict, # classification function >> batch_predict
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=1000) # size of the neighborhood to learn the linear model

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=5, hide_rest=False)
img_boundry1 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry1)

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=10, hide_rest=False)
img_boundry2 = mark_boundaries(temp/255.0, mask)
plt.imshow(img_boundry2)